**Model 2 – Demand-Based Pricing**

In [1]:
# ---------------------------------- MODEL 2: DEMAND-BASED PRICING ----------------------------------

# Install required packages (only once)
!pip install pathway bokeh panel --quiet

# ----------------------------- Imports & Preprocessing -----------------------------
import pandas as pd
import pathway as pw
import bokeh.plotting
import panel as pn

# Load dataset and clean
df = pd.read_csv("dataset.csv")
df.columns = df.columns.str.strip()
df = df.rename(columns={"SystemCodeNumber": "LotID"})

# Map traffic condition: "low", "average", "high" → 1, 2, 3
traffic_map = {"low": 1, "average": 2, "high": 3}
df["TrafficConditionNearby"] = (
    df["TrafficConditionNearby"]
    .astype(str)
    .str.strip()
    .str.lower()
    .map(traffic_map)
    .fillna(1)
)

# Combine timestamp
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                 format="%d-%m-%Y %H:%M:%S")

# Prepare full dataset
df = df[[
    "Timestamp", "LotID", "Occupancy", "Capacity",
    "QueueLength", "TrafficConditionNearby", "IsSpecialDay", "VehicleType"
]]
df.to_csv("parking_stream.csv", index=False)

# ----------------------------- Pathway Schema -----------------------------
class ParkingSchema(pw.Schema):
    Timestamp: str
    LotID: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: float
    IsSpecialDay: int
    VehicleType: str

# Simulate real-time stream
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    LotID = data.LotID
)

# ----------------------------- Model 2 Logic -----------------------------
BASE_PRICE = 10.0
ALPHA = 0.5
BETA = 0.2
GAMMA = 0.1
DELTA = 1.0
EPSILON = 0.8
LAMBDA = 0.6

vehicle_weights = {
    "bike": 0.5,
    "car": 1.0,
    "truck": 1.5
}

@pw.udf
def demand_price(
    occupancy: int,
    capacity: int,
    queue: int,
    traffic: float,
    special_day: int,
    vehicle_type: str
) -> float:
    if capacity == 0:
        return BASE_PRICE
    occ_ratio = occupancy / capacity
    vehicle_weight = vehicle_weights.get(vehicle_type.lower(), 1.0)

    demand = (
        ALPHA * occ_ratio +
        BETA * queue -
        GAMMA * traffic +
        DELTA * special_day +
        EPSILON * vehicle_weight
    )

    norm_demand = (demand - 0.5) / 3
    norm_demand = max(0, min(norm_demand, 1))

    price = BASE_PRICE * (1 + LAMBDA * norm_demand)
    return round(max(5, min(20, price)), 2)

price_stream = data_with_time.with_columns(
    price = demand_price(
        data_with_time.Occupancy,
        data_with_time.Capacity,
        data_with_time.QueueLength,
        data_with_time.TrafficConditionNearby,
        data_with_time.IsSpecialDay,
        data_with_time.VehicleType
    )
)

# ----------------------------- Bokeh Plot -----------------------------
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400, width=800, title="Model 2: Demand-Based Pricing",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="purple")
    fig.circle("t", "price", source=source, size=6, color="orange")
    return fig

viz = price_stream.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [3]:
import sys
import contextlib

with contextlib.redirect_stdout(sys.stdout):
    pw.run()

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input      finished                                     
    PythonRead…     finished          7309        18274    output        11111       None                          
                                                                                                                   
                                                            Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    